# Compare Sentinel-2 and Venus


 This notebook demonstrates querying the STAC API for a Venus and Sentinel-2 chip using an ROI, calculate the NDVI and plot it as a comparison

## Import and Init Environment

In [ ]:
# Import

from glob import glob
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pystac_client import Client
from matplotlib import pyplot as plt
import geopandas as gpd
import requests
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from odc.stac import configure_rio, stac_load

Authenticate with the STAC API, if you get an error check your credentials in `.env`

In [ ]:
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
load_dotenv()  # take environment variables from .env.

def get_new_token():
    auth_server_url = os.getenv("EDS_AUTH_URL")
    client_id = os.getenv("EDS_CLIENT_ID")
    client_secret = os.getenv("EDS_SECRET")
    token_req_payload = {'grant_type': 'client_credentials'}

    token_response = requests.post(
        auth_server_url,
        data=token_req_payload,
        verify=False,
        allow_redirects=False,
        auth=(client_id, client_secret)
    )
    token_response.raise_for_status()

    tokens = json.loads(token_response.text)
    return tokens['access_token']

token = get_new_token()

catalog = Client.open(os.getenv("EDS_API_URL"), headers={
    "Authorization": f"bearer {token}"
})

Matplotlib is building the font cache; this may take a moment.


FileNotFoundError: Credentials file /Users/matthieuhyrien/.earthdaily/credentials not found. Make sure the path is valid

## Define the config

In [ ]:
# define config
datetime = ['2019-11','2020-05']
assets = ['blue','green','red','nir']
clear_cover = 20
geom = earthdaily.datasets.load_pivot_corumba() # or past any wkt here

## Generate and load the Sentinel-2 datacube

In [ ]:
# Sentinel-2 datacube
datacube_s2 = eds.datacube("sentinel-2-l2a",
             datetime=datetime,
             assets=assets,
             intersects=geom,
             mask_with='ag_cloud_mask',
             clear_cover=clear_cover)
datacube_s2.attrs['sensor'] = "Sentinel-2-l2a"
datacube_s2 = datacube_s2.load()


## Generate and load the Venus datacube

In [ ]:
# Venus datacube
datacube_venus = eds.datacube("venus-l2a",
             datetime=datetime,
             assets=assets,
             intersects=geom,
             mask_with='native',
             geobox=datacube_s2.odc.geobox, #same resolution and projection as S2
             clear_cover=clear_cover)
datacube_venus.attrs['sensor'] = "Venus-l2a"
venus = datacube_venus.load()

# Results (plots)

## Compare the NDVI

In [ ]:

datacube_s2.mean(('x','y')).ed.add_indices(['NDVI'])['NDVI'].plot(ls='-',marker='o', label='Sentinel-2')
datacube_venus.mean(('x','y')).ed.add_indices(['NDVI'])['NDVI'].plot(ls='-',marker='o', label='Venus')
plt.legend()
plt.title('NDVI between Sentinel-2 and Venus')

## Compare RGB time series

In [ ]:
for sensor in [datacube_s2, datacube_venus]:
    sensor.ed.plot_rgb()
    plt.suptitle(sensor.attrs['sensor'])
    plt.show()